In [ ]:
!pip install mlxtend
!pip install efficient-apriori

In [1]:
import warnings, random

import datetime
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

import efficient_apriori as efa
from mlxtend.frequent_patterns import apriori, association_rules, fpmax, hmine, fpcommon, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm  # Para visualizar o progresso

warnings.filterwarnings('ignore')

In [2]:
path = 'D:/Disco/Data/PUCRS/proj CDIA III/'

tables = ['PUC_VENDAS', 'PUC_CAMPANHAS']

In [3]:
vendas_dataset = pq.ParquetDataset(f'{path}/dataset/{tables[0]}')
#campanhas_dataset = pq.ParquetDataset(f'{path}/dataset/{tables[1]}')

In [113]:
vendas = vendas_dataset.read_pandas()
#campanhas = campanhas_dataset.read_pandas()

vendas = vendas.to_pandas()

In [114]:
vendas['COD_CUPOM_LOJA'] = vendas['COD_CUPOM'].astype(str) + '-' + vendas['COD_LOJA'].astype(str)

max_cod = vendas['COD_CLIENTE'].max()
mask_zero = (vendas['COD_CLIENTE'] == 0) | (pd.isna(vendas['COD_CLIENTE']))
loc_cupom = vendas.loc[mask_zero, 'COD_CUPOM_LOJA']
unique_cod_cupom = loc_cupom.unique()
cod_clientes = np.arange(max_cod + 1, max_cod + len(unique_cod_cupom) + 1, dtype=int)

map_codes = dict(zip(unique_cod_cupom, cod_clientes))
vendas.loc[mask_zero, 'COD_CLIENTE'] = vendas.loc[mask_zero, 'COD_CUPOM_LOJA'].map(map_codes)

vendas['COD_CLIENTE'] = vendas['COD_CLIENTE'].astype(int)

In [115]:
cupons_list = vendas['COD_CUPOM_LOJA'].unique()
vendas_train, vendas_test =  train_test_split(cupons_list, test_size=.2, random_state=36)

vendas_train = vendas[vendas['COD_CUPOM_LOJA'].isin(vendas_train)]
vendas_test = vendas[vendas['COD_CUPOM_LOJA'].isin(vendas_test)]

In [ ]:
transaction_data = buildTransactionList(vendas_train, transaction_type='customer', min_size=2)

In [116]:
products = produtos = vendas.groupby(['COD_SKU', 'SKU', 'CATEGORIA_SKU', 'SUBCATEGORIA_SKU'], as_index=False).agg({'PRECO_REGULAR': 'mean'})
products.rename(columns={'PRECO_REGULAR': 'PRECO_REGULAR_AVG'}, inplace=True)
products

,COD_SKU,SKU,CATEGORIA_SKU,SUBCATEGORIA_SKU,PRECO_REGULAR_AVG
0,7,SERVICO EM DOMICILIO,DIVERSOS,SERVICOS TELE ENTREGA,5.000000
1,10,MANIPULADOS,MANIPULADOS,MANIPULADOS MEDICAMENTOS,0.010000
2,11,SERVICO DE APLICACAO DE INJETAVEIS,SERVIÇOS,SERVICOS FARMACEUTICOS,5.000000
3,12,SERVICO DE GESTO VACINAL GRIPE CONVENIOS,SERVIÇOS,SERVICOS FARMACEUTICOS,7.983931
4,13,SERVICO DE VERIFICACAO DE GLICEMIA CAPIL,SERVIÇOS,SERVICOS FARMACEUTICOS,3.000000
...,...,...,...,...,...
20653,100027853,ESC CAB MARCO BONI 7,PERFUMARIA,PERFUMARIA,14.990000
20654,100027855,ESC CAB MARCO BONI 8,PERFUMARIA,PERFUMARIA,29.990000
20655,100027878,NEBULIZADOR PULMOMAI,MEDICAMENTOS,LIBERADOS,219.900000
20656,100027882,AMPOLA PANTENE 3UN NUTRE,PERFUMARIA,PERFUMARIA,27.956292


In [195]:
def extract_rules_eff(transactions_df, min_support, min_threshold):
    transactions = [tuple(set(transaction)) for transaction in transactions_df.tolist()]
    itemsets, rules = efa.apriori(transactions, min_support=min_support, min_confidence=min_threshold)

    temp_df = []
    for rule in rules:
        rl = {
            'lhs':rule.lhs,
            'rhs':rule.rhs,
            'confidence':rule.confidence,
            'conviction':rule.conviction,
            'lift':rule.lift,
            'rpf':rule.rpf,
            'support':rule.support
        }

        temp_df.append(rl)

    temp_df = pd.DataFrame(temp_df)

    del transactions, itemsets, rules

    return temp_df    

def extract_rules(transactions_df, min_support, min_threshold, metric):
    
    # Assuming 'transaction_data' is a list of lists where each sublist contains SKUs bought in a transaction.
    te = TransactionEncoder()
    te_ary = te.fit(transactions_df).transform(transactions_df)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    # Apply the apriori algorithm with a minimum support of 0.01 (adjustable)
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)
    #frequent_itemsets = fpgrowth(df_encoded, min_support=min_support)

    # Generate association rules with a minimum confidence of 0.5 (adjustable)
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)

    del te, te_ary, df_encoded, frequent_itemsets

    return rules

'''
def get_suggestions(sku_list, rules_df):
    
    if not isinstance(rules_df['antecedents'].iloc[0], (list, tuple, set)):
        rules_df['antecedents'] = rules_df['antecedents'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    filtered_rules = rules_df[rules_df['antecedents'].apply(lambda x: any(sku in x for sku in sku_list))]
    
    sorted_rules = filtered_rules.sort_values(by=['consequent support', 'confidence'], ascending=False)

    # Obter todos os SKUs sugeridos da coluna 'consequents'
    suggested_skus = sorted_rules['consequents'].explode().unique()
    
    # Selecionar no máximo 6 SKUs
    suggested_skus = suggested_skus[:6]
    
    return suggested_skus
'''
    
def get_suggestions(sku_list, rules_df, top_n=15):

    suggestions = pd.DataFrame(columns=['rhs', 'confidence', 'conviction', 'lift'])

    filtered_rules = rules_df[rules_df['lhs'].apply(lambda x: any(sku in x for sku in sku_list))]
    filtered_rules = filtered_rules.sort_values(by=['confidence', 'lift', 'conviction'], ascending=False)

    if filtered_rules.empty == False:
        suggestions = filtered_rules[['rhs', 'confidence', 'conviction', 'lift']].copy()
        
        # Remover SKUs já presentes na sku_list
        suggestions = suggestions[~suggestions['rhs'].isin(sku_list)]

        # Expandir o 'rhs' caso contenha múltiplos itens
        suggestions = suggestions.explode('rhs')
        
        # Remover duplicatas baseadas no 'rhs'
        suggestions = suggestions.drop_duplicates(subset='rhs')
        
        # Selecionar apenas os top_n resultados
        suggestions = suggestions.head(top_n)
        
        # Resetar o índice para melhor apresentação
        suggestions = suggestions.reset_index(drop=True)
    
    return suggestions

def get_recommendations(suggestions_df, products_df, discount):
    # Verificar se o desconto está entre 0 e 1
    if not 0 <= discount <= 1:
        raise ValueError("O desconto deve ser um valor decimal entre 0 e 1 (por exemplo, 0.10 para 10%)")
    
    # Mesclar suggestions_df com products_df com base na correspondência de 'rhs' e 'COD_SKU'
    merged_df = suggestions_df.merge(products_df, left_on='rhs', right_on='COD_SKU', how='left')
    
    # Verificar se a mesclagem resultou em produtos correspondentes
    if merged_df.empty:
        #print("Nenhum produto correspondente encontrado para as sugestões fornecidas.")
        return merged_df
    
    # Aplicar o desconto ao preço regular e adicionar nova coluna
    merged_df['PRECO_COM_DESCONTO'] = merged_df['PRECO_REGULAR_AVG'] * (1 - discount)
    
    # Selecionar as colunas desejadas
    columns_to_return = ['COD_SKU', 'SKU', 'CATEGORIA_SKU', 'PRECO_REGULAR_AVG', 'PRECO_COM_DESCONTO']
    recommended_products = merged_df[columns_to_return]
    
    return recommended_products


def evalSuggestions(sales_test, products, rules):
    """
    Avalia o desempenho das recomendações geradas pelas regras de associação.
    
    Parâmetros:
    - sales_test: DataFrame contendo as vendas reais no conjunto de teste.
    - rules: DataFrame contendo as regras de associação.

    Retorna:
    - metrics_df: DataFrame com as métricas de avaliação calculadas.
    """
    # Lista para armazenar as métricas
    metrics_list = []

    # Agrupar as compras por cupom
    grouped_sales = sales_test.groupby('COD_CUPOM_LOJA')

    # Iterar sobre cada cupom
    for cod_cupom, group in tqdm(grouped_sales, desc='Avaliando Recomendações'):
        #for cod_cupom in sales_test.groupby('COD_CUPOM'):

        # Produtos comprados pelo cliente neste período
        purchased_products = group['COD_SKU'].unique().tolist()

        # Simular o carrinho atual (pode ser os produtos comprados anteriormente)
        # Aqui, assumimos que o cliente já comprou alguns produtos e queremos recomendar adicionais
        # Para simplificar, vamos assumir que o carrinho está vazio ou usar os produtos do cliente no conjunto de treinamento
        cart = purchased_products  # Aqui você pode customizar como obter o carrinho do cliente

        # Gerar sugestões com base no carrinho e nas regras
        suggestions = get_suggestions(cart, rules)

        # Obter as recomendações (DataFrame)
        recommendations_df = get_recommendations(suggestions, products, discount=.05)
        recommended_products = recommendations_df['COD_SKU'].tolist()

        # Criar vetores binários para cálculo das métricas
        # Produtos relevantes (1 se o produto foi comprado, 0 caso contrário)
        y_true = [1 if sku in purchased_products else 0 for sku in recommended_products]

        # Produtos recomendados (1 para todos os recomendados)
        y_pred = [1]*len(recommended_products)

        # Se não houver recomendações, pular para evitar divisões por zero
        if not recommended_products:
            continue

        # Calcular métricas
        precision = precision_score(y_true, y_pred, zero_division=0)
        recall = recall_score(y_true, y_pred, zero_division=0)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        accuracy = accuracy_score(y_true, y_pred)

        # Armazenar as métricas
        metrics_list.append({
            'COD_CUPOM': cod_cupom,
            'COD_CLIENTE': group['COD_CLIENTE'].unique()[0],
            'Items':len(purchased_products),
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Accuracy': accuracy,
            'Num_Recommendations': len(recommended_products),
            'Num_Relevant': sum(y_true),
            'purchased_products':purchased_products,
            'recommended_products':recommended_products
        })

    # Converter a lista de métricas em DataFrame
    metrics_df = pd.DataFrame(metrics_list, columns=['COD_CUPOM','COD_CLIENTE','Items','Precision','Recall','F1-Score','Accuracy','Num_Recommendations','Num_Relevant','purchased_products','recommended_products'])

    # Calcular as métricas médias
    avg_metrics = metrics_df[['Precision', 'Recall', 'F1-Score', 'Accuracy']].mean().to_dict()
    print("Métricas Médias:")
    for metric, value in avg_metrics.items():
        print(f"{metric}: {value:.4f}")

    return metrics_df


In [176]:
def buildTransactionList(sales_df, transaction_type='cupom', min_size=1):
    transaction_data = []
    
    if transaction_type == 'cupom':
        transaction_data = sales_df.groupby(by=['COD_CUPOM_LOJA'])['COD_SKU'].apply(list)
    if transaction_type == 'customer':
        transaction_data = sales_df.groupby(by=['COD_CLIENTE'])['COD_SKU'].apply(list)

    transaction_data = transaction_data[transaction_data.apply(lambda x: len(x) >= min_size)]

    return transaction_data

In [ ]:
metric = "confidence"
min_support=0.001
min_threshold = 0.03

# Convert the data into a transactional format where each transaction (COD_CUPOM) lists the products (COD_SKU) purchased
# Grouping the data by 'COD_CUPOM' and aggregating the SKUs purchased in each transaction
# sales_tmp.groupby('COD_CUPOM')['COD_SKU'].apply(list)
transaction_data = buildTransactionList(vendas, transaction_type='customer')
sample_data = transaction_data.sample(frac=.5)
rules = extract_rules(sample_data, min_support=min_support, min_threshold=min_threshold, metric=metric)
print(f'extracted {len(rules)} rules')

In [181]:
metric = "confidence"
min_support=0.0001
min_threshold = 0.1


rules = extract_rules_eff(transaction_data, min_support, min_threshold)
print(f'extracted {len(rules)} rules')

extracted 2998841 rules


In [191]:
rules[rules['confidence'] > .85]

,lhs,rhs,confidence,conviction,lift,rpf,support
7,"(12,)","(10038480,)",0.971311,3.466292e+01,174.317805,0.001204,0.001240
46,"(32,)","(10038480,)",0.997255,3.623031e+02,178.973843,0.004265,0.004277
3089,"(10004473,)","(10004472,)",0.871508,7.780583e+00,3348.199433,0.000178,0.000204
5908,"(10024603,)","(10024602,)",1.000000,9.880397e+08,83.609689,0.000238,0.000238
5909,"(10025083,)","(10024602,)",0.943662,1.753770e+01,78.899284,0.000165,0.000175
...,...,...,...,...,...,...,...
2998673,"(10035291, 10035300, 10035311, 100011752, 1000...","(10035299,)",0.962500,2.631256e+01,72.483967,0.000097,0.000101
2998675,"(10035291, 10035299, 10035300, 100011752, 1000...","(10035311,)",0.855556,6.848189e+00,79.092542,0.000086,0.000101
2998677,"(10035291, 10035299, 10035300, 10035311, 10001...","(100020204,)",0.885057,8.533130e+00,46.143640,0.000089,0.000101
2998679,"(10035291, 10035299, 10035300, 10035311, 10001...","(100020206,)",0.875000,7.842077e+00,44.325545,0.000088,0.000101


In [199]:
cupom_lst = vendas_train['COD_CUPOM_LOJA'].unique()[:500]
evalSuggestions(vendas_train[vendas_train['COD_CUPOM_LOJA'].isin(cupom_lst)], products, rules[rules['confidence'] > .8])

Avaliando Recomendações: 100%|██████████| 500/500 [01:48<00:00,  4.61it/s]

Métricas Médias:
Precision: 0.0063
Recall: 0.0625
F1-Score: 0.0113
Accuracy: 0.0063


,COD_CUPOM,COD_CLIENTE,Items,Precision,Recall,F1-Score,Accuracy,Num_Recommendations,Num_Relevant,purchased_products,recommended_products
0,140687-1001,37954689,1,0.000000,0.0,0.000,0.000000,6,0,[10094214],"[10032109, 10035289, 10035299, 10035291, 10027..."
1,140819-1001,37821690,4,0.000000,0.0,0.000,0.000000,6,0,"[100019526, 10023500, 100018365, 10023505]","[10032109, 10027058, 10026744, 10035289, 10002..."
2,140840-1001,29994602,1,0.000000,0.0,0.000,0.000000,4,0,[10025787],"[10032109, 10027058, 10035299, 100020206]"
3,140981-1001,34037988,1,0.000000,0.0,0.000,0.000000,10,0,[10027059],"[10032109, 10026744, 10027058, 10035289, 10106..."
4,141192-1001,29951507,3,0.000000,0.0,0.000,0.000000,3,0,"[1001860, 10024787, 1001448]","[10032109, 10035299, 10027058]"
...,...,...,...,...,...,...,...,...,...,...,...
219,170653-1001,34119812,5,0.000000,0.0,0.000,0.000000,15,0,"[10102997, 1008347, 10096976, 100012687, 6833]","[10027058, 10032109, 10038480, 10026744, 10035..."
220,170657-1001,34009942,1,0.000000,0.0,0.000,0.000000,1,0,[10032163],[10032109]
221,170838-1001,44242295,4,0.000000,0.0,0.000,0.000000,11,0,"[15163, 10004858, 10031973, 1002472]","[10032109, 10027058, 10035299, 10036714, 10035..."
222,170937-1001,38011461,5,0.066667,1.0,0.125,0.066667,15,1,"[10036714, 10036708, 10019408, 10037928, 10037...","[10038480, 10027058, 10032109, 10026744, 10035..."


In [144]:
transaction_data[transaction_data.index == 120]

COD_CLIENTE
120    [91659, 100018365]
Name: COD_SKU, dtype: object

In [136]:
rules_temp = extract_rules_eff(transaction_data[transaction_data.index == 120], min_support, min_threshold)
rules_temp

,lhs,rhs,confidence,conviction,lift,rpf,support
0,"(100018365,)","(91659,)",1.0,0.0,1.0,1.0,1.0
1,"(91659,)","(100018365,)",1.0,0.0,1.0,1.0,1.0


In [150]:
filtered_rules = rules[rules['lhs'].apply(lambda x: any(sku in x for sku in [91659, 100018365]))]
filtered_rules = filtered_rules.sort_values(by=['support', 'lift', 'confidence', 'conviction'], ascending=False)

filtered_rules

,lhs,rhs,confidence,conviction,lift,rpf,support
30540,"(100018365,)","(10032109,)",0.092978,1.068361,3.001894,0.000031,0.000337
31267,"(100018365,)","(10032398,)",0.087776,1.094321,50.609104,0.000028,0.000318
26085,"(100018365,)","(10027058,)",0.081708,1.062798,3.398689,0.000024,0.000296
39706,"(100018365,)","(100018356,)",0.081058,1.085590,33.703209,0.000024,0.000294
39717,"(100018365,)","(100018360,)",0.080841,1.085735,39.684467,0.000024,0.000293
...,...,...,...,...,...,...,...
31788,"(100018365,)","(10033505,)",0.028392,1.024936,6.818714,0.000003,0.000103
39738,"(100018365,)","(100023058,)",0.028175,1.026685,12.565303,0.000003,0.000102
39740,"(100018365,)","(100026723,)",0.027742,1.026160,12.022118,0.000003,0.000101
27482,"(100018365,)","(10028241,)",0.027742,1.022285,4.566445,0.000003,0.000101


In [152]:
cart = [10032109,91659, 100018365]#, 4264, 100009844, 100027743 
products[products['COD_SKU'].isin(cart)][['COD_SKU', 'SKU', 'PRECO_REGULAR_AVG']]

,COD_SKU,SKU,PRECO_REGULAR_AVG
1426,91659,TINT MAXTON CR 4.0 C,16.990000
10767,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,13.057047
19447,100018365,PICOLE KIBON PALITO,9.119884


In [139]:
suggestions = get_suggestions(cart, rules)
suggestions

,rhs,confidence,conviction,lift
0,10032109,0.819672,5.373695,26.464018
1,10027058,0.773196,4.303092,32.161565
2,10026744,0.640000,2.745691,55.405870
3,10036714,0.583658,2.368626,42.169990
4,10106051,0.474432,1.880352,40.387862
5,100019903,0.409091,1.676439,43.625994
6,10032398,0.087776,1.094321,50.609104
7,100018356,0.081058,1.085590,33.703209
8,100018360,0.080841,1.085735,39.684467
9,100018346,0.059168,1.061100,35.149982


In [140]:
get_recommendations(suggestions, products, discount=.05)[['COD_SKU', 'SKU', 'PRECO_REGULAR_AVG', 'PRECO_COM_DESCONTO']]

,COD_SKU,SKU,PRECO_REGULAR_AVG,PRECO_COM_DESCONTO
0,10032109,PAPEL HIG LOUVRE FOLHA DUPLA 20M 12UN NE,13.057047,12.404195
1,10027058,PAPEL HIG STYLUS FOL,12.988897,12.339452
2,10026744,REFRI COCA COLA PET 2L,11.370968,10.802419
3,10036714,LAVA ROUPAS TIXAN YPE LIQ 900ML PRIMAVER,13.385166,12.715908
4,10106051,REFRI COCA COLA LATA 350ML,4.719614,4.483633
5,100019903,ENERGETICO MONSTER ENERGY GREEN 473ML,10.455598,9.932818
6,10032398,PICOLE KIBON TABLITO MAX AVELA 70ML,9.498324,9.023407
7,100018356,PICOLE KIBON CHICABO,7.610528,7.230002
8,100018360,PICOLE KIBON PALITO,8.616883,8.186039
9,100018346,PICOLE KIBON FRUTTAR,6.605128,6.274871


In [200]:
vendas.columns

Index(['COD_CUPOM', 'COD_CLIENTE', 'CLIENTE_FISICO_JURIDICO', 'SEXO_CLIENTE',
       'DTNASCIMENTO_CLIENTE', 'COD_SKU', 'SKU', 'CATEGORIA_SKU',
       'SUBCATEGORIA_SKU', 'UF_CIDADE', 'COD_CIDADE', 'NOME_CIDADE',
       'DATA_CUPOM', 'UNIDADES', 'IDENTIFICADOR_PROMOCIONAL', 'PRECO_REGULAR',
       'TOTAL_DESCONTO', 'TOTAL_BRUTO', 'TOTAL_LIQUIDO', 'COD_LOJA',
       'COD_CUPOM_LOJA'],
      dtype='object')

In [ ]:
['COD_CLIENTE', 'COD_SKU', 'SKU', 'CATEGORIA_SKU','SUBCATEGORIA_SKU','DATA_CUPOM', 'UNIDADES', 'PRECO_REGULAR','TOTAL_DESCONTO', 'TOTAL_BRUTO', 'TOTAL_LIQUIDO', 'COD_CUPOM_LOJA']

In [66]:
rules.to_csv('rules.csv')

In [ ]:
rules.sort_values(by=['confidence'], ascending=False).head(50)

10035299, 10035289, 10035291

In [ ]:
get_suggestions([12722], rules)

In [ ]:
produtos[produtos['COD_SKU'].isin([12722])]

In [ ]:
#frequent_itemsets 
# Sort frequent itemsets by support in descending order
sorted_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Select the top 20 most frequent items
top_20_items = sorted_itemsets.head(20)

print("Top 20 most frequent items:")
print(top_20_items)

# If you want to visualize these items
plt.figure(figsize=(12, 6))
plt.bar(range(len(top_20_items)), top_20_items['support'])
plt.xticks(range(len(top_20_items)), top_20_items['itemsets'].apply(lambda x: ', '.join(list(x))), rotation=90)
plt.xlabel('Items')
plt.ylabel('Support')
plt.title('Top 20 Most Frequent Items')
plt.tight_layout()
plt.show()

# Select items with support greater than 1%
relevant_items = frequent_itemsets[frequent_itemsets['support'] > 0.01]

print("\nItems with support greater than 1%:")
print(relevant_items)


In [ ]:
df = vendas

# Creating a pivot table where rows are products (SKU), columns are products, and values are the count of co-occurrences
product_matrix = df.groupby(['COD_CUPOM', 'CATEGORIA_SKU'])['UNIDADES'].sum().unstack().fillna(0)

# Creating a correlation matrix to find relationships between products
product_correlation = product_matrix.corr()

# Plotting the heatmap to show relationships between products
plt.figure(figsize=(10, 8))
sns.heatmap(product_correlation, cmap='coolwarm', annot=False)
plt.title('Correlação dos produtos')
plt.show()